In [43]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
import numpy as np


In [44]:
em=pd.read_csv('MMT_donepreprocess.csv')

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
# List of Indonesian stop words
indonesian_malay_stop_words = [
    'yang', 'dan', 'di', 'dari', 'ke', 'untuk', 'dengan', 'pada', 'adalah', 'oleh',
    'sebagai', 'dalam', 'itu', 'ini', 'untuk', 'sebuah', 'juga', 'tidak', 'atau',
    'mereka', 'kami', 'saya', 'kamu', 'dia', 'apa', 'satu', 'dua', 'tiga', 'empat', 
    'lima', 'enam', 'tujuh', 'delapan', 'sembilan', 'sepuluh','yang', 'dan', 'di', 'dari', 'ke', 'untuk', 'dengan', 'pada', 'adalah', 'oleh',
    'sebagai', 'dalam', 'itu', 'ini', 'untuk', 'sebuah', 'juga', 'tidak', 'atau',
    'mereka', 'kami', 'saya', 'anda', 'dia', 'apa', 'satu', 'dua', 'tiga', 'empat', 
    'lima', 'enam', 'tujuh', 'lapan', 'sembilan', 'sepuluh','ni','mcm','mahu','yg','pru','spr',
    'tu','la','dah','nak','dgn','ok','kau'
    
]

In [61]:
tfidf=TfidfVectorizer(max_df=0.95,min_df=2,stop_words=indonesian_malay_stop_words)

In [62]:
dtm=tfidf.fit_transform(em['normalized_text'])

In [63]:
dtm

<6671x4408 sparse matrix of type '<class 'numpy.float64'>'
	with 52131 stored elements in Compressed Sparse Row format>

In [64]:
nmf_model=NMF(n_components=5,random_state=100)
nmf_model.fit(dtm)
for index, topic in enumerate(nmf_model.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{index}")
    feature_names = tfidf.get_feature_names_out()
    print([feature_names[i] for i in topic.argsort()[-15:]])
    print('\n')


THE TOP 15 WORDS FOR TOPIC #0
['beliau', 'wang', 'perdana', 'bangun', 'ambil', 'isu', 'laku', 'selamat', 'tindak', 'menteri', 'ekonomi', 'politik', 'rana', 'malaysia', 'negara']


THE TOP 15 WORDS FOR TOPIC #1
['laksana', 'pn', 'pusat', 'je', 'kes', 'rm', 'covid', 'lantan', 'salah', 'gagal', 'sabah', 'bantu', 'selangor', 'negeri', 'raja']


THE TOP 15 WORDS FOR TOPIC #2
['bodoh', 'asli', 'tua', 'umno', 'muda', 'batu', 'kena', 'je', 'cakap', 'kerja', 'kat', 'politik', 'melayu', 'ramai', 'orang']


THE TOP 15 WORDS FOR TOPIC #3
['politik', 'bebas', 'lepas', 'pimpin', 'pas', 'harap', 'umno', 'menang', 'tanding', 'sokong', 'calon', 'parti', 'undi', 'raya', 'pilih']


THE TOP 15 WORDS FOR TOPIC #4
['percaya', 'sahaja', 'beban', 'umno', 'tipu', 'bn', 'bangkang', 'harap', 'baik', 'wakil', 'khidmat', 'malaysia', 'pimpin', 'bantu', 'rakyat']




In [13]:
topic_results=nmf_model.transform(dtm)
topic_results.argmax(axis=1)

array([2, 0, 0, ..., 0, 0, 3], dtype=int64)

In [14]:
em['Topic']=topic_results.argmax(axis=1)

In [15]:
mytopic_dict={
    0:'Size Availability',
    1:'Comfortability',
    2:'Durability',
    3:'Discount and Promotion',
    4:'Price'}

em['Topic Label']=em['Topic'].map(mytopic_dict)

In [16]:
for index, topic in enumerate(nmf_model.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{index}")
    feature_names = tfidf.get_feature_names_out()
    top_keywords = [feature_names[i] for i in topic.argsort()[-15:]]
    print(top_keywords)
    print('\n')
    em.loc[em['Topic'] == index, 'top_keywords'] = ', '.join(top_keywords)

# Display modified DataFrame
em.head()


THE TOP 15 WORDS FOR TOPIC #0
['beliau', 'wang', 'perdana', 'bangun', 'ambil', 'isu', 'laku', 'selamat', 'tindak', 'menteri', 'ekonomi', 'politik', 'rana', 'malaysia', 'negara']


THE TOP 15 WORDS FOR TOPIC #1
['laksana', 'pn', 'pusat', 'je', 'kes', 'rm', 'covid', 'lantan', 'salah', 'gagal', 'sabah', 'bantu', 'selangor', 'negeri', 'raja']


THE TOP 15 WORDS FOR TOPIC #2
['bodoh', 'asli', 'tua', 'umno', 'muda', 'batu', 'kena', 'je', 'cakap', 'kerja', 'kat', 'politik', 'melayu', 'ramai', 'orang']


THE TOP 15 WORDS FOR TOPIC #3
['politik', 'bebas', 'lepas', 'pimpin', 'pas', 'harap', 'umno', 'menang', 'tanding', 'sokong', 'calon', 'parti', 'undi', 'raya', 'pilih']


THE TOP 15 WORDS FOR TOPIC #4
['percaya', 'sahaja', 'beban', 'umno', 'tipu', 'bn', 'bangkang', 'harap', 'baik', 'wakil', 'khidmat', 'malaysia', 'pimpin', 'bantu', 'rakyat']




C:\Users\Asus\AppData\Local\Temp\ipykernel_13336\2198270135.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'beliau, wang, perdana, bangun, ambil, isu, laku, selamat, tindak, menteri, ekonomi, politik, rana, malaysia, negara' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  em.loc[em['Topic'] == index, 'top_keywords'] = ', '.join(top_keywords)


,text,sentiment,cleaning,case_folding,tokenize,StopWord Removal,stemming_data,normalized_text,Topic,Topic Label,top_keywords
0,perempuan tu apesal aku rasa mcm pernah nampak...,0,perempuan tu apesal aku rasa mcm pernah nampak...,perempuan tu apesal aku rasa mcm pernah nampak...,"['perempuan', 'tu', 'apesal', 'aku', 'rasa', '...","['perempuan', 'tu', 'apesal', 'mcm', 'nampak',...",perempuan tu apesal mcm nampak student unisel ...,perempuan tu apesal mcm nampak student unisel ...,2,Durability,"bodoh, asli, tua, umno, muda, batu, kena, je, ..."
1,Polis tangkap dia.,-1,Polis tangkap dia,polis tangkap dia,"['polis', 'tangkap', 'dia']","['polis', 'tangkap']",polis tangkap,polis tangkap,0,Size Availability,"beliau, wang, perdana, bangun, ambil, isu, lak..."
2,Kenapa lokasi kebakaran nya terlalu spesifik? ...,-1,Kenapa lokasi kebakaran nya terlalu spesifik,kenapa lokasi kebakaran nya terlalu spesifik,"['kenapa', 'lokasi', 'kebakaran', 'nya', 'terl...","['lokasi', 'kebakaran', 'nya', 'spesifik']",lokasi bakar nya spesifik,lokasi bakar nya spesifik,0,Size Availability,"beliau, wang, perdana, bangun, ambil, isu, lak..."
3,@YuleumSummer Waa senang nye lah nk bersalin.....,1,Waa senang nye lah nk bersalinape tips alia e...,waa senang nye lah nk bersalinape tips alia e...,"['waa', 'senang', 'nye', 'lah', 'nk', 'bersali...","['waa', 'senang', 'nye', 'nk', 'bersalinape', ...",waa senang nye nk bersalinape tips alia ehshar...,waa senang nye nk bersalinape tips alia ehshar...,2,Durability,"bodoh, asli, tua, umno, muda, batu, kena, je, ..."
4,DAULAT TUANKU! Merafak sembah dan takzim KDYMM...,1,DAULAT TUANKU Merafak sembah dan takzim KDYMM ...,daulat tuanku merafak sembah dan takzim kdymm ...,"['daulat', 'tuanku', 'merafak', 'sembah', 'dan...","['daulat', 'tuanku', 'merafak', 'sembah', 'tak...",daulat tuanku rafak sembah takzim kdymm seri p...,daulat tuanku rafak sembah takzim kdymm seri p...,0,Size Availability,"beliau, wang, perdana, bangun, ambil, isu, lak..."


In [17]:
#DATA PREPROCESSING
df= pd.DataFrame(em[['Topic','Topic Label','top_keywords']])
df.head(5)

,Topic,Topic Label,top_keywords
0,2,Durability,"bodoh, asli, tua, umno, muda, batu, kena, je, ..."
1,0,Size Availability,"beliau, wang, perdana, bangun, ambil, isu, lak..."
2,0,Size Availability,"beliau, wang, perdana, bangun, ambil, isu, lak..."
3,2,Durability,"bodoh, asli, tua, umno, muda, batu, kena, je, ..."
4,0,Size Availability,"beliau, wang, perdana, bangun, ambil, isu, lak..."


In [65]:
# Coherence Score Calculation
# Prepare data for gensim
texts = [text.split() for text in em['normalized_text']]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Create NMF topics
topics = [[feature_names[i] for i in topic.argsort()[-15:]] for topic in nmf_model.components_]
coherence_model = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

In [66]:
# Cohesion Score Calculation
def calculate_cohesion(dtm, topic_results):
    n_topics = topic_results.shape[1]
    cohesion_scores = []
    for topic in range(n_topics):
        topic_docs_indices = np.where(topic_results.argmax(axis=1) == topic)[0]
        if len(topic_docs_indices) > 1:
            topic_docs = dtm[topic_docs_indices].toarray()
            similarity_matrix = cosine_similarity(topic_docs)
            np.fill_diagonal(similarity_matrix, 0)
            cohesion = np.mean(similarity_matrix)
            cohesion_scores.append(cohesion)
        else:
            cohesion_scores.append(0)  # Assign zero if only one document or none in the topic
    return np.mean(cohesion_scores)

cohesion_score = calculate_cohesion(dtm, topic_results)

In [67]:
print(f"Coherence Score: {coherence_score}")
print(f"Cohesion Score: {cohesion_score}")

Coherence Score: 0.4066919762846699
Cohesion Score: 0.01865359703994563
